# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather = pd.read_csv("../output_data/cities.csv")
weather


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Mahébourg,75,MU,1614818133,94,-20.4081,57.7000,24.00,1.03
1,1,Arraial do Cabo,0,BR,1614818037,83,-22.9661,-42.0278,25.00,6.17
2,2,Ogaminana,68,NG,1614818134,75,7.5946,6.2248,24.36,2.19
3,3,Hilo,1,US,1614817993,44,19.7297,-155.0900,26.00,5.14
4,4,Coquimbo,75,CL,1614817998,88,-29.9533,-71.3436,17.00,3.60
...,...,...,...,...,...,...,...,...,...,...
537,537,Warrnambool,100,AU,1614818086,91,-38.3833,142.4833,15.56,1.34
538,538,Lake Shasta,1,US,1614818243,14,40.6804,-122.3708,23.00,3.60
539,539,Marrakesh,100,MA,1614818243,72,31.6315,-8.0083,14.00,2.06
540,540,Qaqortoq,75,GL,1614818243,34,60.7167,-46.0333,8.00,20.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#Configure gmaps
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight
locations= weather[["Lat", "Lng"]]
humidity= weather["Humidity"]

In [17]:
#heatmap
figure=gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_int=np.max(humidity)

#heatlayer
heat_lay=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3 )

#layer 2
figure.add_layer(heat_lay)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
ideal_df = weather[(weather['Max Temp'].between(1,12)) & (weather['Wind Speed']<=8) & (weather['Humidity']<=45)]
ideal_df.reset_index(inplace=True)
del ideal_df['index']


ideal_df
 

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,142,Rawlins,1,US,1614818161,26,41.7911,-107.2387,7.00,4.12
1,155,Bethel,1,US,1614818165,42,41.3712,-73.4140,7.00,2.57
2,186,Jiayuguan City,8,CN,1614818171,30,39.8167,98.3000,1.19,3.40
3,232,Pincher Creek,1,CA,1614818180,32,49.4833,-113.9520,11.00,7.72
4,333,Brigantine,1,US,1614818201,39,39.4101,-74.3646,8.00,2.06
5,361,Levittown,1,US,1614818128,42,40.1551,-74.8288,8.00,2.55
6,390,Palmer,75,US,1614818042,45,42.1584,-72.3287,5.00,4.63
7,416,Cody,1,US,1614818218,23,44.5263,-109.0565,9.00,4.63
8,419,Bowmore,1,US,1614818219,43,34.9421,-79.3031,12.00,2.04
9,516,Bāglung,0,NP,1614818239,42,28.2719,83.5898,10.94,2.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df=ideal_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Rawlins,US,41.7911,-107.2387,
1,Bethel,US,41.3712,-73.4140,
2,Jiayuguan City,CN,39.8167,98.3000,
3,Pincher Creek,CA,49.4833,-113.9520,
4,Brigantine,US,39.4101,-74.3646,
5,Levittown,US,40.1551,-74.8288,
6,Palmer,US,42.1584,-72.3287,
7,Cody,US,44.5263,-109.0565,
8,Bowmore,US,34.9421,-79.3031,
9,Bāglung,NP,28.2719,83.5898,


In [20]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [21]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Rawlins.
Closest hotel in Rawlins is Fairfield Inn & Suites by Marriott Rawlins.
------------
Retrieving Results for Index 1: Bethel.
Closest hotel in Bethel is Courtyard by Marriott Danbury.
------------
Retrieving Results for Index 2: Jiayuguan City.
Closest hotel in Jiayuguan City is Jiayuguan Hotel.
------------
Retrieving Results for Index 3: Pincher Creek.
Closest hotel in Pincher Creek is Heritage Inn Hotel & Convention Centre.
------------
Retrieving Results for Index 4: Brigantine.
Closest hotel in Brigantine is Ocean Casino Resort.
------------
Retrieving Results for Index 5: Levittown.
Closest hotel in Levittown is Sheraton Bucks County Langhorne.
------------
Retrieving Results for Index 6: Palmer.
Closest hotel in Palmer is The Trainmaster's Inn.
------------
Retrieving Results for Index 7: Cody.
Closest hotel in Cody is Best Western Premier Ivy Inn & Suites.
------------
Retrieving Results for Index 8: Bowmore.
Missing field/result... skipp

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)
# Display figure
figure



Figure(layout=FigureLayout(height='420px'))